
# Projet: Facebook BaBi tasks
Notebook de la semaine du 22/03
Par Thierry Loesch et Bryce Tichit

Comme demandé par notre tuteur, nous allons désormais essayer d'adapter les travaux fait jusqu'ici sur un autre projet sensiblement identique.

-----------------------------------

5.1  Notation automatique de réponses courtes à des questions

Les données sont ici pour le téléchargement et leur description est dans cet article

Lorsque l'on connait:

    une question,
    une réponse d'un étudiant,
    la bonne réponse, 

comment prédire la note à donné à la réponse de l'étudiant. Il peut être aussi intéressant d'essayer d'apprendre à générer la réponse. 

--------------------------

On cherchera en premier à prédire la note de l'étudiant, le réseau devra rendre un réel. Normaliser les notes (les mettre entre 0 et 1) et activation par Sigmoid (output entre 0 et 1) afin de prédire la note.


TODO:

- Parser données
- Vectorisation
- Modèle

In [24]:
import re
from keras.utils.data_utils import get_file
import zipfile
import string

def tokenize(sent):
    
    def remPunctuation(sent):
        return "".join(l for l in sent if l not in string.punctuation)
        
    return [x.strip() for x in re.split('(\W+)?', remPunctuation(sent)) if x.strip()]

def parseData(questions,answers,student_answers=''):
    
    #questions_data = dict() #ce dictionnaire contiendra les tuples (question,answer) indexés par la clé "numéro de question"
    
    questions_data = dict()
    
    for line in questions.split('\n'): #une ligne = une question
        line = line.decode('utf-8').strip()
        try:
            index,question = line.split(' ',1)
        except:
            continue
        
        questions_data[index] = [question.replace('<STOP>',''),None]
    
    for line in answers.split('\n'):
        line = line.decode('utf-8').strip()
        
        try:
            index,answer = line.split(' ',1)
        except:
            continue
            
        questions_data[index][1] = answer.replace('<STOP>','')
        
    questions_data = {k: tuple(map(tokenize,v)) for k, v in questions_data.items()}
    
    stud_ans = dict()
    
    for line in student_answers.split('\n'):
        line = line.decode('utf-8')
        try:
            index,ans = line.split(' ',1)
        except:
            continue
            
        answers=ans.split('<STOP>')
        answers=[a for a in answers if a]
            
        if index in stud_ans:
            stud_ans[index] += map(tokenize,answers)
        else:
            stud_ans[index] = map(tokenize,answers)
    
    return questions_data,stud_ans
        
    
def parseIntoExamples(qd,sa):
    
    ret = list()
    
    for index,q_a in qd.items():
        for stud_ans in sa[index]:
            ret.append((q_a[0],q_a[1],stud_ans))
    return ret
                       

try:
    path = get_file('shortAnswerGrading-v-2-0.zip',origin='http://web.eecs.umich.edu/~mihalcea/downloads/ShortAnswerGrading_v2.0.zip')
except:
    print "error while downloading file"
    
    
archive = zipfile.ZipFile(path,'r')
    
questions = archive.read('data/sent/questions') 
answers = archive.read('data/sent/answers')
student_answers = archive.read('data/sent/all') 

dic1,dic2 = parseData(questions,answers,student_answers)


ret = parseIntoExamples(dic1,dic2)
    

In [27]:
ret[0]

([u'What', u'is', u'a', u'function', u'pointer'],
 [u'The',
  u'address',
  u'of',
  u'the',
  u'location',
  u'in',
  u'memory',
  u'where',
  u'the',
  u'function',
  u'code',
  u'resides'],
 [u'A',
  u'pointer',
  u'that',
  u'points',
  u'to',
  u'the',
  u'memory',
  u'address',
  u'of',
  u'the',
  u'beginning',
  u'part',
  u'of',
  u'a',
  u'functions',
  u'pretty',
  u'much',
  u'the',
  u'pointer',
  u'points',
  u'to',
  u'the',
  u'function',
  u'so',
  u'it',
  u'can',
  u'be',
  u'called',
  u'to',
  u'execute'])